In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#es para correrlo en colab
from google.colab import files
import io
#finn

from lightgbm import LGBMClassifier
import xgboost as xgb

from sklearn.preprocessing import LabelEncoder

In [ ]:
test = pd.read_csv("data/test.csv")
ensemble_submit = test['id'].to_frame()

## CNN

In [ ]:
from   keras                        import initializers, regularizers, constraints, callbacks, optimizers
from   keras.layers                 import Conv1D, Embedding, GlobalMaxPooling1D, concatenate, Input, Dense,Dropout
from   keras.models                 import Sequential, Model
from   keras.preprocessing.sequence import pad_sequences
from   keras.preprocessing.text     import Tokenizer
from   sklearn.model_selection      import train_test_split
from   gensim.models.keyedvectors   import KeyedVectors

### Preparacion de datos

In [ ]:
train_cnn = pd.read_csv('data/tran_cnn.csv')
test_cnn = pd.read_csv('data/test_cnn.csv')

In [ ]:
test_cnn.text_clean.fillna(" ", inplace=True)
test_it=test_cnn[["id","text_clean"]]
total_words = train_cnn[["id","text_clean"]]
total_words=pd.concat([total_words,test_it], sort = 'False')

In [ ]:
numWords=20000
tokenizer = Tokenizer(num_words = numWords)
tokenizer.fit_on_texts(total_words.text_clean)
sequences_train  = tokenizer.texts_to_sequences(train_cnn['text_clean'])
sequences_test  =  tokenizer.texts_to_sequences(test_cnn['text_clean'])
word_index       = tokenizer.word_index

In [ ]:
max_length       = 300
x_train          = pad_sequences(sequences_train, maxlen = max_length)
x_test           = pad_sequences(sequences_test, maxlen = max_length)
x_features_train = train_cnn[['len_text', 'count_word_uniques',"text_contain_keyword_norm","contain_question_norm","contain_hashtag_norm"]]
x_features_test  = test_cnn[['len_text', 'count_word_uniques',"text_contain_keyword_norm","contain_question_norm","contain_hashtag_norm"]]
y_train          = train_cnn["target"].values

In [ ]:
word_vectors = KeyedVectors.load_word2vec_format('https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz', binary = True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
embeddind_dimension = 300
vocabulary_size = min(len(word_index) + 1, numWords)
embedding_matrix = np.zeros((vocabulary_size, embeddind_dimension))

In [ ]:
for word, i in word_index.items():
    if i < numWords:
        try:
            embedding_vector = word_vectors[word]
            embedding_matrix[i] = embedding_vector
        except KeyError:
            embedding_matrix[i] = np.random.normal(0, np.sqrt(0.25), embeddind_dimension)

del(word_vectors)

In [ ]:
e = Embedding(len(embedding_matrix), 200, input_length=300)

In [ ]:
Xtrain1, Xval1, ytrain1, yval1 = train_test_split(x_train, y_train,
                                              train_size = 0.95, random_state = 233)

Xfeaturestrain1, Xfeaturesval1, ytrain1, yval1 = train_test_split(x_features_train, y_train,
                                                            train_size = 0.95, random_state = 233)

### Uso de modelos

In [ ]:
model = Sequential()
model.add(e)
model.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.25))
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 300, 200)          4000000   
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 299, 100)          40100     
_________________________________________________________________
global_max_pooling1d_3 (Glob (None, 100)               0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 256)               25856     
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 257       
Total params: 4,066,213
Trainable params: 4,066,213
Non-trainable params: 0
____________________________________________

In [ ]:
model.fit(Xtrain1, ytrain1,
          validation_data=(Xval1, yval1), 
          epochs=5, 
          batch_size=300,
          verbose=1)

Epoch 1/5
25/25 [==============================] - 16s 636ms/step - loss: 0.4167 - accuracy: 0.8494 - val_loss: 0.5952 - val_accuracy: 0.7690
Epoch 2/5
25/25 [==============================] - 16s 626ms/step - loss: 0.0508 - accuracy: 0.9835 - val_loss: 1.0852 - val_accuracy: 0.7559
Epoch 3/5
25/25 [==============================] - 16s 628ms/step - loss: 0.0343 - accuracy: 0.9847 - val_loss: 1.1655 - val_accuracy: 0.7533
Epoch 4/5
25/25 [==============================] - 16s 635ms/step - loss: 0.0309 - accuracy: 0.9855 - val_loss: 1.1684 - val_accuracy: 0.7638
Epoch 5/5
25/25 [==============================] - 16s 639ms/step - loss: 0.0306 - accuracy: 0.9841 - val_loss: 1.1513 - val_accuracy: 0.7690


In [ ]:
y_pred1 = model.predict_classes(x_test)

In [ ]:
model3 = Sequential()
model3.add(e)
model3.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
model3.add(GlobalMaxPooling1D())
model3.add(Dense(200, activation='relu'))
model3.add(Dropout(0.25))
model3.add(Dense(200, activation='relu'))
model3.add(Dense(100, activation='relu'))
model3.add(Dense(1, activation='sigmoid'))
model3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model3.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 300, 200)          4000000   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 299, 100)          40100     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 200)               20200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 200)               40200     
_________________________________________________________________
dense_4 (Dense)              (None, 100)              

In [ ]:
model3.fit(x = [Xtrain1, Xfeaturestrain1], y = ytrain1, epochs=5,
          validation_data = ([Xval1, Xfeaturesval1], yval1), verbose = 1)

Epoch 1/5
226/226 [==============================] - 27s 119ms/step - loss: 0.2356 - accuracy: 0.9086 - val_loss: 0.6483 - val_accuracy: 0.7769
Epoch 2/5
226/226 [==============================] - 27s 118ms/step - loss: 0.0845 - accuracy: 0.9730 - val_loss: 0.7708 - val_accuracy: 0.7612
Epoch 3/5
226/226 [==============================] - 26s 117ms/step - loss: 0.0503 - accuracy: 0.9815 - val_loss: 0.8233 - val_accuracy: 0.7533
Epoch 4/5
226/226 [==============================] - 27s 118ms/step - loss: 0.0342 - accuracy: 0.9848 - val_loss: 1.1825 - val_accuracy: 0.7375
Epoch 5/5
226/226 [==============================] - 26s 117ms/step - loss: 0.0292 - accuracy: 0.9837 - val_loss: 1.2188 - val_accuracy: 0.7533


In [ ]:
y_pred3 = model3.predict_classes([x_test, x_features_test])

In [ ]:
model4 = Sequential()
model4.add(e)
model4.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
model4.add(GlobalMaxPooling1D())
model4.add(Dense(200, activation='relu'))
model4.add(Dropout(0.25))
model4.add(Dense(200, activation='relu'))
model4.add(Dense(100, activation='relu'))
model4.add(Dense(50, activation='relu'))
model4.add(Dropout(0.25))
model4.add(Dense(1, activation='sigmoid'))
model4.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model4.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 300, 200)          4000000   
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 299, 100)          40100     
_________________________________________________________________
global_max_pooling1d_4 (Glob (None, 100)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 200)               20200     
_________________________________________________________________
dropout_4 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 200)               40200     
_________________________________________________________________
dense_12 (Dense)             (None, 100)              

In [ ]:
model4.fit(x = [Xtrain1, Xfeaturestrain1], y = ytrain1, epochs=5,
          validation_data = ([Xval1, Xfeaturesval1], yval1), verbose = 1)

Epoch 1/5
226/226 [==============================] - 27s 121ms/step - loss: 0.0974 - accuracy: 0.9705 - val_loss: 1.0184 - val_accuracy: 0.7690
Epoch 2/5
226/226 [==============================] - 27s 118ms/step - loss: 0.0388 - accuracy: 0.9833 - val_loss: 1.1153 - val_accuracy: 0.7480
Epoch 3/5
226/226 [==============================] - 27s 117ms/step - loss: 0.0273 - accuracy: 0.9853 - val_loss: 1.4452 - val_accuracy: 0.7480
Epoch 4/5
226/226 [==============================] - 26s 117ms/step - loss: 0.0244 - accuracy: 0.9841 - val_loss: 2.4974 - val_accuracy: 0.7533
Epoch 5/5
226/226 [==============================] - 26s 117ms/step - loss: 0.0277 - accuracy: 0.9856 - val_loss: 1.4893 - val_accuracy: 0.7559


In [ ]:
y_pred4 = model4.predict_classes([x_test, x_features_test])

In [ ]:
submitFinal = test['id'].to_frame()
submitFinal["pred1"]=y_pred1
submitFinal["pred2"]=y_pred3
submitFinal["pred3"]=y_pred4
submitFinal["suma"]=submitFinal.pred1+submitFinal.pred2+submitFinal.pred3
submitFinal["target"]=submitFinal.suma.apply(lambda x: 1 if x>2 else 0)

In [ ]:
y_cnn=submitFinal["target"]

## Light GBM

In [ ]:
train = pd.read_csv("data/train_pre_processing_true.csv")
test = pd.read_csv("data/test_pre_processing_true.csv")
x_train, y_train = train.select_dtypes(include=['float64','int64','bool']).iloc[:,:-1],train.iloc[:,-1]
x_test = test.drop(columns=['id']).select_dtypes(include=['float64','int64','bool'])
light_model = LGBMClassifier(random_state=1)
light_model.fit(x_train, y_train)
preds_1 = light_model.predict(x_test)
ensemble_submit['target_lgbm_pred1'] = preds_1

NameError: ignored

In [ ]:
train = pd.read_csv("data/train_pre_processing_false.csv")
test = pd.read_csv("data/test_pre_processing_false.csv")
x_train, y_train = train.select_dtypes(include=['float64','int64','bool']).iloc[:,:-1],train.iloc[:,-1]
x_train.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in x_train.columns]
x_test = test.drop(columns=['id']).select_dtypes(include=['float64','int64','bool'])
x_test.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in x_test.columns]
light_model = LGBMClassifier(random_state=1)
light_model.fit(x_train, y_train)
preds_2 = light_model.predict(x_test)
ensemble_submit['target_lgbm_pred2'] = preds_2

NameError: ignored

In [ ]:
train = pd.read_csv("data/train_pre_processing_true_false.csv")
test = pd.read_csv("data/test_pre_processing_true_false.csv")
x_train, y_train = train.select_dtypes(include=['float64','int64','bool']).iloc[:,:-1],train.iloc[:,-1]
x_train.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in x_train.columns]
x_test = test.drop(columns=['id']).select_dtypes(include=['float64','int64','bool'])
x_test.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in x_test.columns]
light_model = LGBMClassifier(random_state=1)
light_model.fit(x_train, y_train)
preds_3 = light_model.predict(x_test)
ensemble_submit['target_lgbm_pred3'] = preds_3

In [ ]:
preds_total = pd.DataFrame({'uno':preds_1,'dos':preds_2,'tres':preds_3})

In [ ]:
ensemble_submit['target_lgbm'] = preds_total.apply(lambda x: (x.uno and x.dos) or (x.tres and x.dos) or (x.uno and x.tres),axis=1)

## XGBoost

In [ ]:
train = pd.read_csv('data/train_pre_processing_nlp_5000.csv')
test = pd.read_csv('data/test_pre_processing_nlp_5000.csv')
train.drop(labels=['id'], axis=1, inplace=True)
test.drop(labels=['id'], axis=1, inplace=True)

In [ ]:
noise_cols = ['location','len_location_cero_default', 
             'total_words_location_cero_default',
             'total_words_location_mean_default', 
              'text']
train.drop(labels=noise_cols, axis=1, inplace=True)
test.drop(labels=noise_cols, axis=1, inplace=True)

In [ ]:
def prepare_df_for_fit(df):
    columns_str = ['keyword', 'text_clean', 'keyword_grouped']
    
    # Encode with LabelEncoder
    encoded_cols = df[columns_str]
    encoded_cols = encoded_cols.astype('str')
    encoded_cols = encoded_cols.apply(LabelEncoder().fit_transform)
    encoded_drop = df.drop(columns_str, axis = 1)
    encoded_df = pd.concat([encoded_drop, encoded_cols], axis = 1)
    # Drop Target column
    if 'target' in encoded_df.columns:
        encoded_df.drop(axis=1, labels=['target'], inplace=True)

    return encoded_df

In [ ]:
train_X = prepare_df_for_fit(train)
test_X = prepare_df_for_fit(test) 
train_Y = train['target']

In [ ]:
xgb_model = xgb.XGBClassifier(objective="reg:linear", random_state=10, colsample_bytree = 0.5, 
                             gamma = 0.1, learning_rate = 0.06, max_depth = 5, min_child_weight = 2, 
                             n_estimators = 300, reg_alpha = 0.1, seed = 123, subsample = 0.9)
xgb_model.fit(train_X, train_Y)
preds = xgb_model.predict(test_X)
ensemble_submit['target_xgb'] = preds

## Result

In [ ]:
ensemble_submit.info()

In [ ]:
column_list = list(ensemble_submit.columns)
column_list.remove("id")
#ensemble_submit["sum"] = ensemble_submit[column_list].sum(axis=1)
ensemble_submit["general_sum"] = ensemble_submit[['target_lgbm','target_xgb']].sum(axis=1)
ensemble_submit.head(5)

In [ ]:
ensemble_submit["target"] = ensemble_submit.general_sum.apply(lambda x: 1 if x >= 2 else 0)

In [ ]:
ensemble_submit = ensemble_submit[['id',"target"]]
ensemble_submit.to_csv('submit_xgb_lgbm_diff_dataset.csv', index=False)